## Helen Gaskell

## Capstone Project:  
### Analysing a dataset of H&M transactions to make personalised recommendations for customers

### Notebook 1: Loading and Cleaning Data

In [1]:
import numpy as np
import pandas as pd

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
import statsmodels.api as sm
from scipy import stats
from scipy.stats import norm

H&M Group is a family of brands and businesses with 53 online markets and approximately 4,850 stores. The online store offers shoppers an extensive selection of products to browse through. But with too many choices, customers might not quickly find what interests them or what they are looking for, and ultimately, they might not make a purchase. To enhance the shopping experience, product recommendations are key. More importantly, helping customers make the right choices also has a positive implications for sustainability, as it reduces returns, and thereby minimizes emissions from transportation.

The data sets contain transactions from both online and in store. We need to bear in mind that the pandemic started around March 2019 which will significantly effect the number of transactions which were bought in store as many stores had to temporarily close down


Starting with 3 tables:

- Transactions: details of each purchase
- Customers: details of every customer
- Articles: details of each product

Transactions Dataset: https://drive.google.com/file/d/1So5F6WkQvdZza3zaT1YeLBnAZZprNCqO/view?usp=share_link

Customers Dataset: https://drive.google.com/file/d/1Maf3J1WGfRyHJDa6jzQ9XBZXVH47qyuz/view?usp=share_link

Articles Dataset: https://drive.google.com/file/d/10j5jPppv8miBI_xvD1ZXz9EgaPmjW5ms/view?usp=share_link

## 1. Transaction data

### Loading Data

In [4]:
# load the data
rawtransaction = pd.read_csv('../src/data/transactions_train.csv.zip')

In [5]:
rawtransaction.shape

(31788324, 5)

Transaction table is 31 million rows by 5 columns 

In [6]:
rawtransaction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31788324 entries, 0 to 31788323
Data columns (total 5 columns):
 #   Column            Dtype  
---  ------            -----  
 0   t_dat             object 
 1   customer_id       object 
 2   article_id        int64  
 3   price             float64
 4   sales_channel_id  int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 1.2+ GB


In [7]:
# Checking for null values 
rawtransaction.isna().sum()

t_dat               0
customer_id         0
article_id          0
price               0
sales_channel_id    0
dtype: int64

There are no null cell values 

In [8]:
# having a look at the table layout 
rawtransaction

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2
...,...,...,...,...,...
31788319,2020-09-22,fff2282977442e327b45d8c89afde25617d00124d0f999...,929511001,0.059305,2
31788320,2020-09-22,fff2282977442e327b45d8c89afde25617d00124d0f999...,891322004,0.042356,2
31788321,2020-09-22,fff380805474b287b05cb2a7507b9a013482f7dd0bce0e...,918325001,0.043203,1
31788322,2020-09-22,fff4d3a8b1f3b60af93e78c30a7cb4cf75edaf2590d3e5...,833459002,0.006763,1


### Creating a sample data set 

In [11]:
#using a sample of dataset to perform initial cleaning and EDA, which can then be implemented on main dataset
transactionsample = rawtransaction.sample(frac=0.01)

In [12]:
transactionsample.shape

(317883, 5)

Sample data set has 300,000 rows 

In [13]:
# quick view of dataframe
transactionsample.head()

,t_dat,customer_id,article_id,price,sales_channel_id
20970547,2020-01-05,6881f635c5be0550646522a81a07d942ee71fcf603397d...,735622003,0.037271,2
22088558,2020-02-06,dea80c33dba2ad05511052e042e446466e7c20a524fe51...,806071001,0.084729,2
19962633,2019-12-08,dfe7bf62e4bedaccc92346586e427fd2eb3d6b2c8d90d5...,781688003,0.076254,2
31448321,2020-09-12,e5774d749808f6c828ace801c925706026ef9b7426a575...,904017001,0.067780,2
23614172,2020-03-22,5480047400bdaaf4a23842785fdda0f4ff53a47b59ee01...,834431001,0.031881,2


### Transaction Data Dictionary

- t_dat - transaction date (object, should change into datetime format) 

- customer_id - unique customer id (string of letters and numbers)

- article_id - unique article id (9 number integer)

- price - price of transaction (float)

- sales_channel_id - 1 is purchase in store, 2 is purchase online 

In [14]:
# looking at date range of the data set 
first_day = transactionsample['t_dat'].min()
last_day = transactionsample['t_dat'].max()

In [15]:
first_day

'2018-09-20'

In [16]:
last_day

'2020-09-22'

Data ranges from September 2018 to September 2020

In [17]:
# checking datatypes
transactionsample.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 317883 entries, 20970547 to 8804914
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   t_dat             317883 non-null  object 
 1   customer_id       317883 non-null  object 
 2   article_id        317883 non-null  int64  
 3   price             317883 non-null  float64
 4   sales_channel_id  317883 non-null  int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 14.6+ MB


### Converting t_dat into Datetime datatype

In [18]:
# converting date column into datetime format
transactionsample["t_dat"] = transactionsample["t_dat"].astype("datetime64")

In [19]:
# checking it's been converted
transactionsample.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 317883 entries, 20970547 to 8804914
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   t_dat             317883 non-null  datetime64[ns]
 1   customer_id       317883 non-null  object        
 2   article_id        317883 non-null  int64         
 3   price             317883 non-null  float64       
 4   sales_channel_id  317883 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(1)
memory usage: 14.6+ MB


In [22]:
rawtransaction['price'].describe()

count    3.178832e+07
mean     2.782927e-02
std      1.918113e-02
min      1.694915e-05
25%      1.581356e-02
50%      2.540678e-02
75%      3.388136e-02
max      5.915254e-01
Name: price, dtype: float64

Price is in float format and ranges from 0.000 to 0.59. H&M have hidden the actual prices so there is no currency/unit, they have likely scaled the values. I will look into this further in the EDA notebook. The max price is quite a lot higher than the mean which could suggest that there are outliers.

In [19]:
# Checking unique values in sales_channel_id
transactionsample['sales_channel_id'].unique()

array([2, 1])

Sales Channel id consists of values 2 and 1, 2 is online, 1 is in-store

In [20]:
# Checking for duplicates 
rawtransaction.duplicated().sum()

2974905

3 million duplicates in original dataset. This is a lot of duplicates therefore I will need to make sure they are intentional and won't skew the results when I reach the modeling stage

In [23]:
# investigating the duplicates further to compare the duplicates against each other
rawtransaction.loc[rawtransaction.duplicated() , :]

,t_dat,customer_id,article_id,price,sales_channel_id
15,2018-09-20,000aa7f0dc06cd7174389e76c9e132a67860c5f65f9706...,501820043,0.016932,2
18,2018-09-20,000aa7f0dc06cd7174389e76c9e132a67860c5f65f9706...,671505001,0.033881,2
20,2018-09-20,000aa7f0dc06cd7174389e76c9e132a67860c5f65f9706...,631848002,0.033881,2
21,2018-09-20,000aa7f0dc06cd7174389e76c9e132a67860c5f65f9706...,631848002,0.033881,2
22,2018-09-20,000aa7f0dc06cd7174389e76c9e132a67860c5f65f9706...,631848002,0.033881,2
...,...,...,...,...,...
31788268,2020-09-22,ff54fe15368409b7d0d691cdeebe2d6a3fee5178105277...,885315003,0.050102,2
31788269,2020-09-22,ff54fe15368409b7d0d691cdeebe2d6a3fee5178105277...,885315003,0.050102,2
31788282,2020-09-22,ff6f55a51af284b71dcd264396b299e548f968c1769e71...,919786002,0.042356,2
31788292,2020-09-22,ff94f31e864d9b655643ac4d2adab3611c7241adb5d34c...,901666001,0.084729,2


In [24]:
# selecting a particular customer id to compare the duplicated rows for that customer
rawtransaction[
    rawtransaction['customer_id'] == "000aa7f0dc06cd7174389e76c9e132a67860c5f65f970699daccc14425ac31a8"]

,t_dat,customer_id,article_id,price,sales_channel_id
14,2018-09-20,000aa7f0dc06cd7174389e76c9e132a67860c5f65f9706...,501820043,0.016932,2
15,2018-09-20,000aa7f0dc06cd7174389e76c9e132a67860c5f65f9706...,501820043,0.016932,2
16,2018-09-20,000aa7f0dc06cd7174389e76c9e132a67860c5f65f9706...,674681001,0.008458,2
17,2018-09-20,000aa7f0dc06cd7174389e76c9e132a67860c5f65f9706...,671505001,0.033881,2
18,2018-09-20,000aa7f0dc06cd7174389e76c9e132a67860c5f65f9706...,671505001,0.033881,2
19,2018-09-20,000aa7f0dc06cd7174389e76c9e132a67860c5f65f9706...,631848002,0.033881,2
20,2018-09-20,000aa7f0dc06cd7174389e76c9e132a67860c5f65f9706...,631848002,0.033881,2
21,2018-09-20,000aa7f0dc06cd7174389e76c9e132a67860c5f65f9706...,631848002,0.033881,2
22,2018-09-20,000aa7f0dc06cd7174389e76c9e132a67860c5f65f9706...,631848002,0.033881,2
23,2018-09-20,000aa7f0dc06cd7174389e76c9e132a67860c5f65f9706...,680187001,0.016932,2


In [24]:
transactionduplicates = rawtransaction.loc[rawtransaction.duplicated() , :]

In [25]:
transactionduplicates.groupby(['t_dat', 'customer_id', 'article_id', 'price', 'sales_channel_id']).count()

Empty DataFrame
Columns: []
Index: [(2018-09-20, 000aa7f0dc06cd7174389e76c9e132a67860c5f65f970699daccc14425ac31a8, 377277001, 0.008457627118644, 2), (2018-09-20, 000aa7f0dc06cd7174389e76c9e132a67860c5f65f970699daccc14425ac31a8, 501820043, 0.0169322033898305, 2), (2018-09-20, 000aa7f0dc06cd7174389e76c9e132a67860c5f65f970699daccc14425ac31a8, 553139001, 0.0338813559322033, 2), (2018-09-20, 000aa7f0dc06cd7174389e76c9e132a67860c5f65f970699daccc14425ac31a8, 631848002, 0.0338813559322033, 2), (2018-09-20, 000aa7f0dc06cd7174389e76c9e132a67860c5f65f970699daccc14425ac31a8, 640639001, 0.0101525423728813, 2), (2018-09-20, 000aa7f0dc06cd7174389e76c9e132a67860c5f65f970699daccc14425ac31a8, 664421002, 0.0169322033898305, 2), (2018-09-20, 000aa7f0dc06cd7174389e76c9e132a67860c5f65f970699daccc14425ac31a8, 671505001, 0.0338813559322033, 2), (2018-09-20, 000aa7f0dc06cd7174389e76c9e132a67860c5f65f970699daccc14425ac31a8, 676827002, 0.0423559322033898, 2), (2018-09-20, 000aa7f0dc06cd7174389e76c9e132a67860c5f65f970699daccc14425ac31a8, 680912006, 0.0169322033898305, 2), (2018-09-20, 000aa7f0dc06cd7174389e76c9e132a67860c5f65f970699daccc14425ac31a8, 680912009, 0.0118474576271186, 2), (2018-09-20, 000aa7f0dc06cd7174389e76c9e132a67860c5f65f970699daccc14425ac31a8, 685687002, 0.0169322033898305, 2), (2018-09-20, 000aa7f0dc06cd7174389e76c9e132a67860c5f65f970699daccc14425ac31a8, 692454002, 0.0254067796610169, 2), (2018-09-20, 000aa7f0dc06cd7174389e76c9e132a67860c5f65f970699daccc14425ac31a8, 700819006, 0.0423559322033898, 2), (2018-09-20, 002b3c0a44a22c45a8d62ea9d2b88d1a89e335f8b8400335a85c3b5845794cb9, 464277014, 0.0220169491525423, 2), (2018-09-20, 0045c79125b4dc958579f902b49eacd8598f9eeaa1220506d1cae1275f122084, 211143037, 0.0338813559322033, 2), (2018-09-20, 0045c79125b4dc958579f902b49eacd8598f9eeaa1220506d1cae1275f122084, 503729006, 0.0423559322033898, 2), (2018-09-20, 00ead79936275fbd3c9cf7863d2eb1b11e553789c48149fec39eb4086cb64642, 678086003, 0.0254067796610169, 2), (2018-09-20, 012bbedf2efe728a7407a5dc842a852f8e09e9ae9727110c916ae13ebbc86395, 651456003, 0.0169322033898305, 1), (2018-09-20, 0132d21d866697c428e4d6ffb02548fd71966aa88c8f5adea314e2341f8ad87b, 608939002, 0.0406610169491525, 2), (2018-09-20, 014cbfc591254f0cf30df52b660f94dead60c8d3eafc00ebb3051bd196bdcd8e, 399136061, 0.0508305084745762, 2), (2018-09-20, 019169333a46eafc04a2a49f04076aeb5bc7f33c403e8940e39048a9581ce753, 632020002, 0.0135423728813559, 2), (2018-09-20, 01a9a077a83eacc6fd946a60c4ca2aed16219fcd83d31e3110fa04630c42b96d, 574120001, 0.0271016949152542, 2), (2018-09-20, 01a9a077a83eacc6fd946a60c4ca2aed16219fcd83d31e3110fa04630c42b96d, 578816004, 0.0304915254237288, 2), (2018-09-20, 01bf0506fe8bb4b830a2b0a4459f11565f6b687c615008cdcd59b3abf6df4bd2, 573716039, 0.0254067796610169, 2), (2018-09-20, 01bf0506fe8bb4b830a2b0a4459f11565f6b687c615008cdcd59b3abf6df4bd2, 680374001, 0.0338813559322033, 2), (2018-09-20, 01bf0506fe8bb4b830a2b0a4459f11565f6b687c615008cdcd59b3abf6df4bd2, 690478001, 0.0338813559322033, 2), (2018-09-20, 01d106a0b45ae7f169b3233d332bf325c80c24d4d160edeccf7d70905077fb86, 573085020, 0.0327457627118644, 2), (2018-09-20, 01d106a0b45ae7f169b3233d332bf325c80c24d4d160edeccf7d70905077fb86, 632143001, 0.0327457627118644, 2), (2018-09-20, 01d106a0b45ae7f169b3233d332bf325c80c24d4d160edeccf7d70905077fb86, 677848003, 0.0081694915254237, 2), (2018-09-20, 01ef5256e6264e9d50b11a08e9cfb2f2291324fedb4140cf7b4ad070548ffd8f, 660497003, 0.008457627118644, 2), (2018-09-20, 01ef5256e6264e9d50b11a08e9cfb2f2291324fedb4140cf7b4ad070548ffd8f, 661308001, 0.0152372881355932, 2), (2018-09-20, 01f0af0b062a155ce270aa4cd7420795b115dd9320c143eaa10213f13ba346be, 728162002, 0.008457627118644, 1), (2018-09-20, 01f597f5eba83f9709eceb5a70a99f3a4009a6e827bf7293afa4211030a29fa4, 456163026, 0.0338813559322033, 2), (2018-09-20, 01f597f5eba83f9709eceb5a70a99f3a4009a6e827bf7293afa4211030a29fa4, 625773001, 0.0593050847457627, 2), (2018-09-20, 01f597f5eba83f9709eceb5a70a99f3a4009a6e827bf7293afa4211030a29fa4, 674336001, 0

It's likely that the duplicates correspond to customers purchasing more than one of the same product. Within the datasets there are no columns which specify the size of an item. It's likely that items have the same article id per size of that item. This would explain the large amount of duplicates. It may also just show the items that customers buy more than one of, which will be useful when considering a baseline model of recommending repeated purchases to customers. It's a good indicator of types of products that customers buy more than one of.

In [31]:
# saving the duplicate table to a new variable dup_df
dup_df = rawtransaction[rawtransaction.duplicated()]
dup_df.head()

,t_dat,customer_id,article_id,price,sales_channel_id
15,2018-09-20,000aa7f0dc06cd7174389e76c9e132a67860c5f65f9706...,501820043,0.016932,2
18,2018-09-20,000aa7f0dc06cd7174389e76c9e132a67860c5f65f9706...,671505001,0.033881,2
20,2018-09-20,000aa7f0dc06cd7174389e76c9e132a67860c5f65f9706...,631848002,0.033881,2
21,2018-09-20,000aa7f0dc06cd7174389e76c9e132a67860c5f65f9706...,631848002,0.033881,2
22,2018-09-20,000aa7f0dc06cd7174389e76c9e132a67860c5f65f9706...,631848002,0.033881,2


In [27]:
# further investigation of duplicate at idx 15
idx = 15
rawtransaction[(rawtransaction.t_dat == dup_df.loc[idx].t_dat) & 
                (rawtransaction.customer_id == dup_df.loc[idx].customer_id) & 
                (rawtransaction.article_id == dup_df.loc[idx].article_id) & 
                (rawtransaction.price == dup_df.loc[idx].price) & 
                (rawtransaction.sales_channel_id == dup_df.loc[idx].sales_channel_id)
               ]

,t_dat,customer_id,article_id,price,sales_channel_id
14,2018-09-20,000aa7f0dc06cd7174389e76c9e132a67860c5f65f9706...,501820043,0.016932,2
15,2018-09-20,000aa7f0dc06cd7174389e76c9e132a67860c5f65f9706...,501820043,0.016932,2


Now I will create a new table called cleantransaction which groups the article id per customer to signify how many times they bought each product. This column will be useful when modeling

In [30]:
cleantransaction = rawtransaction.groupby(rawtransaction.columns.tolist()).size().reset_index().rename(columns={0:'article_purchase_count'})
cleantransaction.sample(10)

,t_dat,customer_id,article_id,price,sales_channel_id,article_purchase_count
12959302,2019-07-23,9f233b838cd7274c8876e7a4143de2352d839f2179aa82...,767687001,0.030492,2,1
8024675,2019-04-17,669e4bc123674f7a1788e6229add13f9db946d7b9b75dd...,733749001,0.005068,1,1
5409129,2019-02-08,657b2538b535d75a89e18cbfe89a09e2ed65eaf20a336a...,663516001,0.067780,2,1
12067144,2019-07-05,5ab52f682bcbbcee573bfdeb8a902b17878f60ce9b9347...,746170001,0.015000,1,1
26302469,2020-07-15,62c6078330fad184ff626f82d09a45e7ea2189c90a8344...,841383003,0.008458,2,1
12521700,2019-07-14,b9bb8e69b3c4a1d9b429c644b48c403d9ff8af5cdb8b00...,720861001,0.046458,2,1
6093349,2019-02-26,acf2fdb32caf924fa5b7695809eed78ef77a3da2ab7952...,370594017,0.013136,2,1
3794769,2018-12-24,c57de756989c61f9aec47c3a4a832bbc25dff314e4e2c1...,610822021,0.006763,2,1
9583753,2019-05-22,3e94c4e71dbe14cbabf5357e1f0924f9f2becb0588783a...,723347001,0.025407,2,2
15577761,2019-09-28,bd6210120b74000c77e4ffd8fe1e55f7edd5c7e1684010...,777143001,0.040661,2,1


In [32]:
# sanity check that new column has different values
cleantransaction['article_purchase_count'].nunique()

75

In [30]:
cleantransaction["t_dat"] = cleantransaction["t_dat"].astype("datetime64")

In [38]:
cleantransaction.groupby('customer_id')['article_purchase_count'].sum()/cleantransaction['customer_id'].nunique()

customer_id
00000dbacae5abe5e23885899a1fa44253a17956c6d1c3d25f88aa139fdfc657    1.541532e-05
0000423b00ade91418cceaf3b26c6af3dd342b51fd051eec9c12fb36984420fa    6.312941e-05
000058a12d5b43e67d225668fa1f8d618c13dc232df0cad8ffe7ad4a1091e318    1.321313e-05
00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2c5feb1ca5dff07c43e    1.468126e-06
00006413d8573cd20ed7128e53b7b13819fe5cfc2d801fe7fc0f26dd8d65a85a    9.542818e-06
                                                                        ...     
ffffbbf78b6eaac697a8a5dfbfd2bfa8113ee5b403e4747568cac33e8c541831    3.743721e-05
ffffcd5046a6143d29a04fb8c424ce494a76e5cdf4fab53481233731b5c4f8b7    6.166129e-05
ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1778d0116cffd259264    3.303283e-05
ffffd7744cebcf3aca44ae7049d2a94b87074c3d4ffe38b2236865d949d4df6a    5.138441e-06
ffffd9ac14e89946416d80e791d064701994755c3ab686a1eaf3458c36f52241    7.340629e-07
Name: article_purchase_count, Length: 1362281, dtype: float64

There are 1,362,281 different customers 



In [32]:
cleantransaction.head()

,t_dat,customer_id,article_id,price,sales_channel_id,article_purchase_count
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2,1
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2,1
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221001,0.020322,2,1
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2,1
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687001,0.016932,2,1


In [33]:
cleantransaction['article_id'].nunique()

104547

104,000 different articles purchased in the 2 year period

### 2. Customer Data

In [39]:
# Load the customer data
rawcustomer = pd.read_csv('../src/data/customers.csv.zip')

In [40]:
rawcustomer.head()

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,NaN,NaN,ACTIVE,NONE,49.0,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,NaN,NaN,ACTIVE,NONE,25.0,2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,NaN,NaN,ACTIVE,NONE,24.0,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,NaN,NaN,ACTIVE,NONE,54.0,5d36574f52495e81f019b680c843c443bd343d5ca5b1c2...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,1.0,1.0,ACTIVE,Regularly,52.0,25fa5ddee9aac01b35208d01736e57942317d756b32ddd...


In [42]:
# checking for null values in customer table 
rawcustomer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1371980 entries, 0 to 1371979
Data columns (total 7 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   customer_id             1371980 non-null  object 
 1   FN                      476930 non-null   float64
 2   Active                  464404 non-null   float64
 3   club_member_status      1365918 non-null  object 
 4   fashion_news_frequency  1355971 non-null  object 
 5   age                     1356119 non-null  float64
 6   postal_code             1371980 non-null  object 
dtypes: float64(3), object(4)
memory usage: 73.3+ MB


In [41]:
rawcustomer.tail()

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
1371975,ffffbbf78b6eaac697a8a5dfbfd2bfa8113ee5b403e474...,NaN,NaN,ACTIVE,NONE,24.0,7aa399f7e669990daba2d92c577b52237380662f36480b...
1371976,ffffcd5046a6143d29a04fb8c424ce494a76e5cdf4fab5...,NaN,NaN,ACTIVE,NONE,21.0,3f47f1279beb72215f4de557d950e0bfa73789d24acb5e...
1371977,ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1...,1.0,1.0,ACTIVE,Regularly,21.0,4563fc79215672cd6a863f2b4bf56b8f898f2d96ed590e...
1371978,ffffd7744cebcf3aca44ae7049d2a94b87074c3d4ffe38...,1.0,1.0,ACTIVE,Regularly,18.0,8892c18e9bc3dca6aa4000cb8094fc4b51ee8db2ed14d7...
1371979,ffffd9ac14e89946416d80e791d064701994755c3ab686...,NaN,NaN,PRE-CREATE,NONE,65.0,0a1a03306fb2f62164c2a439b38c0caa64b40deaae8687...


### Data Dictionary

- customer_id - Unique identifier of every customer (same column as in transactions table)
- FN - If customer gets a fashion newsletter (1 or Nan)
- Active - If customer is active for communication (1 or Nan)
- club_member_status - member or non-member (active, Nan, pre-create or left club)
- fashion_news_frequency - How often H&M send news to the customer (none, regularly, Nan, Monthly or None)
- age - Current age of customer
- Postal_code - Current postcode of customer (actual postcodes are hidden)

Actual location details of postal code have been hidden therefore the only thing we can deduce from this column is if different transactions have been made from exactly the same postcode 

In [43]:
rawcustomer['FN'].unique()

array([nan,  1.])

In [44]:
rawcustomer['Active'].unique()

array([nan,  1.])

In [45]:
rawcustomer['club_member_status'].unique()

array(['ACTIVE', nan, 'PRE-CREATE', 'LEFT CLUB'], dtype=object)

In [45]:
rawcustomer['fashion_news_frequency'].unique()

array(['NONE', 'Regularly', nan, 'Monthly', 'None'], dtype=object)

In [46]:
rawcustomer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1371980 entries, 0 to 1371979
Data columns (total 7 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   customer_id             1371980 non-null  object 
 1   FN                      476930 non-null   float64
 2   Active                  464404 non-null   float64
 3   club_member_status      1365918 non-null  object 
 4   fashion_news_frequency  1355971 non-null  object 
 5   age                     1356119 non-null  float64
 6   postal_code             1371980 non-null  object 
dtypes: float64(3), object(4)
memory usage: 73.3+ MB


1,371,980 customer rows 

This is compared to 1,362,281 in the transactions table, suggesting that some of the customers in this table haven't made any purchases.

0.7% of customers missing in transaction data

Around 65% of FN and Active columns are Null values

0.5% nulls in club_member_status

1% nulls in fashion_news_frequency

1% nulls in age

No nulls in Postal_code or customer_id

In [47]:
rawcustomer.duplicated().sum()

0

There are no customer duplicates 

### Dealing with null values

In [48]:
rawcustomer.isnull().sum()

customer_id                    0
FN                        895050
Active                    907576
club_member_status          6062
fashion_news_frequency     16009
age                        15861
postal_code                    0
dtype: int64

There are only no null values in postal_code and customer_id

In [49]:
customersample = rawcustomer.sample(frac=0.25)

In [50]:
customersample.shape

(342995, 7)

In [46]:
# dropping FN column as mentioned earlier, 65% of the values are NaN so the column won't
# be very useful to us
# saving to new variable cleancustomer
cleancustomer = rawcustomer.drop(['FN'], axis=1)

In [52]:
rawcustomer['age'].mean()

36.386964565794

In [47]:
# filling age null values with mean age
cleancustomer['age']= rawcustomer['age'].fillna(rawcustomer['age'].mean())

In [48]:
rawcustomer['club_member_status'].isnull().sum()

6062

In [49]:
rawcustomer['club_member_status'].value_counts()

ACTIVE        1272491
PRE-CREATE      92960
LEFT CLUB         467
Name: club_member_status, dtype: int64

In [50]:
# replacing NaN values in club_member_status with unknown 
cleancustomer['club_member_status'] = rawcustomer['club_member_status'].replace(np.NaN, 'UNKNOWN')

In [51]:
rawcustomer['Active'].value_counts()

1.0    464404
Name: Active, dtype: int64

In [52]:
# filling null values in Active with 0
cleancustomer['Active'] = rawcustomer['Active'].fillna(0)

In [53]:
rawcustomer['fashion_news_frequency'].value_counts()


NONE         877711
Regularly    477416
Monthly         842
None              2
Name: fashion_news_frequency, dtype: int64

In [54]:
# replacing null values and 'None' with 'UNKNOWN'
cleancustomer['fashion_news_frequency'] = rawcustomer['fashion_news_frequency'].replace('None', 'UNKNOWN')
cleancustomer['fashion_news_frequency'] = rawcustomer['fashion_news_frequency'].replace('NONE', 'UNKNOWN')
cleancustomer['fashion_news_frequency'] = rawcustomer['fashion_news_frequency'].replace(np.NaN, 'UNKNOWN')

### Converting data types

Converting age and active column to int

In [55]:
cleancustomer['age'] = cleancustomer['age'].astype(int)
cleancustomer['Active'] = cleancustomer['Active'].astype(int)

### 3. Articles Data

In [56]:
rawproduct = pd.read_csv('../src/data/articles.csv.zip')

In [57]:
rawproduct.head(15).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
article_id,108775015,108775044,108775051,110065001,110065002,110065011,111565001,111565003,111586001,111593001,111609001,112679048,112679052,114428026,114428030
product_code,108775,108775,108775,110065,110065,110065,111565,111565,111586,111593,111609,112679,112679,114428,114428
prod_name,Strap top,Strap top,Strap top (1),OP T-shirt (Idro),OP T-shirt (Idro),OP T-shirt (Idro),20 den 1p Stockings,20 den 1p Stockings,Shape Up 30 den 1p Tights,Support 40 den 1p Tights,200 den 1p Tights,SWEATSHIRT OC,SWEATSHIRT OC,Alice BANDEAU 2-p,Alice BANDEAU 2-p
product_type_no,253,253,253,306,306,306,304,302,273,304,304,252,252,306,306
product_type_name,Vest top,Vest top,Vest top,Bra,Bra,Bra,Underwear Tights,Socks,Leggings/Tights,Underwear Tights,Underwear Tights,Sweater,Sweater,Bra,Bra
product_group_name,Garment Upper body,Garment Upper body,Garment Upper body,Underwear,Underwear,Underwear,Socks & Tights,Socks & Tights,Garment Lower body,Socks & Tights,Socks & Tights,Garment Upper body,Garment Upper body,Underwear,Underwear
graphical_appearance_no,1010016,1010016,1010017,1010016,1010016,1010016,1010016,1010016,1010016,1010016,1010016,1010001,1010001,1010017,1010016
graphical_appearance_name,Solid,Solid,Stripe,Solid,Solid,Solid,Solid,Solid,Solid,Solid,Solid,All over pattern,All over pattern,Stripe,Solid
colour_group_code,9,10,11,9,10,12,9,13,9,9,9,7,71,10,6
colour_group_name,Black,White,Off White,Black,White,Light Beige,Black,Beige,Black,Black,Black,Grey,Light Blue,White,Light Grey


In [64]:
rawproduct.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105542 entries, 0 to 105541
Data columns (total 25 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   article_id                    105542 non-null  int64 
 1   product_code                  105542 non-null  int64 
 2   prod_name                     105542 non-null  object
 3   product_type_no               105542 non-null  int64 
 4   product_type_name             105542 non-null  object
 5   product_group_name            105542 non-null  object
 6   graphical_appearance_no       105542 non-null  int64 
 7   graphical_appearance_name     105542 non-null  object
 8   colour_group_code             105542 non-null  int64 
 9   colour_group_name             105542 non-null  object
 10  perceived_colour_value_id     105542 non-null  int64 
 11  perceived_colour_value_name   105542 non-null  object
 12  perceived_colour_master_id    105542 non-null  int64 
 13 

In [65]:
rawproduct.head()

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
2,108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
3,110065001,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,9,Black,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
4,110065002,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,10,White,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."


105,542 entries x 
24 columns 

416 null values in detail_desc

0.4% of products have missing descriptions. This is a low number therefore should be okay to drop as there are descriptions of the items in the other columns

Dropping rows with desc null vals 

In [58]:
# dropping null values in description and saving to new variable cleanarticles
cleanarticles = rawproduct.dropna().reset_index(drop=True)


In [59]:
cleanarticles.head()

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
2,108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
3,110065001,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,9,Black,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
4,110065002,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,10,White,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."


In [60]:
cleanarticles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105126 entries, 0 to 105125
Data columns (total 25 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   article_id                    105126 non-null  int64 
 1   product_code                  105126 non-null  int64 
 2   prod_name                     105126 non-null  object
 3   product_type_no               105126 non-null  int64 
 4   product_type_name             105126 non-null  object
 5   product_group_name            105126 non-null  object
 6   graphical_appearance_no       105126 non-null  int64 
 7   graphical_appearance_name     105126 non-null  object
 8   colour_group_code             105126 non-null  int64 
 9   colour_group_name             105126 non-null  object
 10  perceived_colour_value_id     105126 non-null  int64 
 11  perceived_colour_value_name   105126 non-null  object
 12  perceived_colour_master_id    105126 non-null  int64 
 13 

### Data Dictionary

- article_id - Unique identifier of every article
- product_code - Unique identifier of type of product
- prod_name -  Name of product e.g. bandeau bra 
- product_type_no - Product sub category number
- product_type_name - Product sub category name e.g. bra 
- product_group_name - Product category name e.g. underwear 
- graphical_appearance_no - group of graphics number
- graphical_appearance_name - group of graphics name e.g. stripey
- colour_group_code - colour code
- colour_group_name - colour name 
- perceived_colour_value_id - Perceived colour number
- perceived_colour_value_name - Dark/Light/Dusty light/Bright/Medium
- perceived_colour_master_id - Perceived master colour number
- perceived_colour_master_name - Perceived master colour name 
- department_no - Department number
- department_name - Department name e.g. Jersey Basic
- index_code - Index number
- index_name - Womenswear/Baby etc 
- index_group_no - Subcategory of Index number
- index_group_name - Subcategory of Index name 
- section_no - Section number
- section_name - Section name 
- garment_group_no - Garment group number
- garment_group_name - Garment group name 
- detail_desc - description of article 

No duplicated rows 

In [69]:
articlesample = rawproduct.sample(frac=0.3)

In [70]:
articlesample.shape

(31663, 25)

In [71]:
cleanarticles.head()

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
2,108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
3,110065001,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,9,Black,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
4,110065002,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,10,White,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."


In [72]:
cleancustomer.head()

,customer_id,Active,club_member_status,fashion_news_frequency,age,postal_code
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0,ACTIVE,NONE,49,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0,ACTIVE,NONE,25,2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0,ACTIVE,NONE,24,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0,ACTIVE,NONE,54,5d36574f52495e81f019b680c843c443bd343d5ca5b1c2...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,1,ACTIVE,Regularly,52,25fa5ddee9aac01b35208d01736e57942317d756b32ddd...


In [73]:
cleantransaction.head()

,t_dat,customer_id,article_id,price,sales_channel_id,article_purchase_count
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2,1
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2,1
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221001,0.020322,2,1
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2,1
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687001,0.016932,2,1


Saving clean datasets to csv file in data folder 

In [74]:
cleanarticles.to_csv('../src/data/cleanarticles.csv')

In [79]:
cleancustomer.to_csv('../src/data/cleancustomers.csv')

In [80]:
cleantransaction.to_csv('../src/data/cleantransactions.csv')